# Exercise 3 - Emotion Recognition
## Convoluted Feelings


### Data
For this exercise, you will be working on the Twitter Emotion Recognition task. The goal of this task is to infer the
affectual state of a person from their tweet. You will be using the Tweeteval dataset to train your model. You can
access the related GitHub repositories from the links below.

● Tweeteval Repository: https://github.com/cardiffnlp/tweeteval

● Emotion Detection (our task): https://github.com/cardiffnlp/tweeteval/tree/main/datasets/emotion

The ability to process and load custom datasets for your projects/research is an important skill. Most tasks you will
complete as an NLP practitioner/researcher will require you to handle different data sources, formats, and files.
Therefore, we want you to figure out how to load the emotion recognition dataset from its respective repository
using all the necessary files (e.g., train_text.txt, train_labels.txt, mapping.txt) and train your model.

### Task: Emotion Recognition with a CNN
Implement an emotion recognition classifier in PyTorch or PyTorch Lightning. You can reuse the class structure
from exercise 2, which is an adaption from Rao and McMahan. However, you are free to create your own, new
class structure. Keep in mind that for emotion prediction, unlike in Exercise 1, we work on the word level instead of
the character level. Thus, your Vocabulary class (if you have one) will not hold a vocabulary of characters.
Remember to document your code with docstrings and/or comments and/or text cells.
1. Pick two different emotion classes for your model to predict (e.g., anger and joy). Load/filter your
dataset to include only the related class data. Create another dataset and change only one of the
classes (e.g., anger and sadness) this time.
2. Your goal is to find the optimal model architecture and training regime for your CNN classifier. Pick one
of the datasets you created and start experimenting. Experiment with at least three different combinations
(sets) of hyperparameters, with at least two different values each, e.g., optimizer, learning rate, dropout,
number of filters, stride, kernel size, pooling, and batch size. Report the combinations and corresponding
results (accuracy and F1-macro) on the development set in a table:
3. Use your best-performing model settings to train another model on the second dataset. Report
the model performance (accuracy and F1-macro) on the test set of both datasets.
4. What could be the reason that the specific combination/values of hyperparameters resulted in the best
model performance? Don’t worry about the exact reasoning, the goal is just to provide educated (or wellreasoned) guesses.

In [1]:
!pip install spacy

In [282]:
import os
import re
import random
import time
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import spacy
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.utils.data as data, torchvision as tv
import torch.optim as optim

In [283]:
nlp = spacy.blank('en')

### Read data
Reading each lines in the data and it's corresponding labels below.

In [320]:
def read_text(text_path):
  with open(text_path, 'r') as file:
        text_data = file.read().splitlines()
  return text_data

def read_labels(label_path):
  with open(label_path, 'r') as file:
        label_data = file.read().splitlines()
  return label_data

## Load Train, Validation, Test data
Loading train, validation, and test dataset and labels

In [321]:
train_txt_file = "train_text.txt"
train_label_file = "train_labels.txt"
train_data = read_text(train_txt_file)
train_labels = read_labels(train_label_file)

val_txt_file = "val_text.txt"
val_label_file = "val_labels.txt"
val_data = read_text(val_txt_file)
val_labels = read_labels(val_label_file)

test_txt_file = "test_text.txt"
test_label_file = "test_labels.txt"
test_data = read_text(test_txt_file)
test_labels = read_labels(test_label_file)

### Example data
Example of how data and labels look like. Mappings are as follows:

0 ---> Anger

1 ---> Joy

2 ---> Optimism

3 ---> Sadness

In [322]:
train_data[0], train_labels[0]

("“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry ",
 '2')

In [323]:
val_data[0], val_labels[0]

('@user @user Oh, hidden revenge and anger...I rememberthe time,she rebutted you. ',
 '0')

In [324]:
test_data[0], test_labels[0]

('#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse ',
 '3')

In [325]:
train_data[0:5]

["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry ",
 "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs ",
 "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu ",
 "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight ",
 "it's pretty depressing when u hit pan on ur favourite highlighter "]

In [326]:
train_labels[0:5]

['2', '0', '1', '0', '3']

### Proprocess and clean text data
1. we decoded ascii values
2. we removed all the tags for eg: @user, @john, etc
3. we remove all the words which were # in it
4. we removed all the puncuations
5. we removed all the stopwords
6. we applied Lemmatization

In [327]:
import nltk
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess and lemmatize text
def preprocess_text(text, clean_hastag_words = False, remove_username = False):
    # Remove unwanted characters
    text = text.encode('ascii', 'ignore').decode('ascii')


    if remove_username:
      text = re.sub(r'@\w+', '', text)
    if clean_hastag_words:
      text = re.sub(r'#\w+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

# Apply preprocessing to each text in the data
train_data = [preprocess_text(text, clean_hastag_words=True, remove_username=True) for text in train_data]
val_data = [preprocess_text(text, clean_hastag_words=True, remove_username=True) for text in val_data]
test_data = [preprocess_text(text, clean_hastag_words=True, remove_username=True) for text in test_data]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Section 1: Emotional Recognition Model: Anger or Joy

In this section, we will choose data from anger (0) and joy (1) classes and train a Emotion Recognition model using only these two classes.

In [328]:
train_d_aj = [data for data, label in zip(train_data, train_labels) if label in ('0', '1')]
train_l_aj = [label for label in train_labels if label in ('0', '1')]

val_d_aj = [data for data, label in zip(val_data, val_labels) if label in ('0', '1')]
val_l_aj = [label for label in val_labels if label in ('0', '1')]

test_d_aj = [data for data, label in zip(test_data, test_labels) if label in ('0', '1')]
test_l_aj = [label for label in test_labels if label in ('0', '1')]

In [329]:
train_l_aj.count("0")

1400

In [330]:
train_l_aj.count("1")

708

###Data balancing

As observed above, our classes are not balanced, therefore we will upsample the minority class using the function below.

In [331]:
def balance_dataset(dataset, labels):
  import random
  # Concatenate the training data and labels
  train_data_labels = list(zip(dataset, labels))

  # Extract minority and majority classes
  minority_class = [(data, label) for data, label in train_data_labels if label == '1']
  majority_class = [(data, label) for data, label in train_data_labels if label == '0']

  # Calculate the required oversampling ratio
  oversampling_ratio = len(majority_class) // len(minority_class)

  # Randomly replicate instances from the minority class
  minority_upsampled = random.choices(minority_class, k=len(majority_class) * oversampling_ratio)

  # Combine minority and majority classes
  upsampled_data_labels = majority_class + minority_upsampled

  # Shuffle the upsampled dataset to ensure randomness
  random.shuffle(upsampled_data_labels)

  # Extract upsampled data and labels
  upsampled_data, upsampled_labels = zip(*upsampled_data_labels)

  return upsampled_data, upsampled_labels

Balance the classes

In [332]:
train_d_aj, train_l_aj =  balance_dataset(train_d_aj, train_l_aj)
val_d_aj, val_l_aj =  balance_dataset(val_d_aj, val_l_aj)
test_d_aj, test_l_aj =  balance_dataset(test_d_aj, test_l_aj)

In [333]:
train_l_aj.count("0")

1400

In [334]:
train_l_aj.count("1")

1400

Examples of filtered data:

In [335]:
train_d_aj[0:10]

('blame whole season natalie season would different turned back alliance',
 'php54 though kardashian stuff go head youre hilarious like gleeful little boy',
 'michelle one worst player bb history',
 'blame jose ere would give rojo another start sunday fucking disaster waiting happen amp shocking header',
 'ffs tate thought wind willow serious play cant wait see play singing badger',
 'thats pretty love sky background purple highlight dull color great',
 'customer service beyond appalling faulty dryer replacement break within wks part 3 wks engineer show',
 'im exited shaking much im pround colleen fandom everyone amazing',
 'ppl talking diet feeling hahaha',
 'control guy')

In [336]:
train_l_aj[0:10]

('0', '1', '0', '0', '1', '1', '0', '1', '1', '0')

In [337]:
val_l_aj[0:10]

('1', '1', '0', '1', '1', '0', '1', '1', '0', '0')

tokenize, encode, converting to tensor functions. Reference of tokenize, encode functions: tutorial notebook

In [338]:
def tokenize(texts):
  max_len = 0
  tokenized_texts = []
  word2idx = {}

  # Add <pad> and <unk> tokens to the vocabulary
  word2idx['<pad>'] = 0
  word2idx['<unk>'] = 1

  # Building our vocab from the corpus starting from index 2
  idx = 2
  for sent in texts:
    tokenized_sent = nlp(sent)
    # Add `tokenized_sent` to `tokenized_texts`
    tokenized_texts.append(tokenized_sent)
    # Add new token to `word2idx`
    for token in tokenized_sent:
      # string any token objects are different things, be careful.
      if token.text not in word2idx:
        word2idx[token.text] = idx
        idx += 1

        # Update `max_len`
    max_len = max(max_len, len(tokenized_sent))

  return tokenized_texts, word2idx, max_len


def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to e
        tokenized_padded_sent = list(tokenized_sent) + ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(str(token)) for token in tokenized_padded_sent]
        input_ids.append(input_id)

    return np.array(input_ids), word2idx


def get_tokenized_encoded_ids(text):
  # get tokenized and encoded data input ids
  tokenized_texts, word2idx, max_len = tokenize(text)
  input_ids, word2idx = encode(tokenized_texts, word2idx, max_len)
  # converting input ids to torch.Tensor
  input_ids = torch.from_numpy(input_ids)
  return input_ids, word2idx

def get_tensor_labels(labels):
  # converting labels to torch.Tensor
  label_int = [int(item) for item in labels]
  label_tensors = torch.tensor(label_int)
  return label_tensors

In [339]:
# tokenizing, encoding, converting to tensors
train_id_aj, word2idx = get_tokenized_encoded_ids(train_d_aj)
val_id_aj, _ = get_tokenized_encoded_ids(val_d_aj)
test_id_aj, _ = get_tokenized_encoded_ids(test_d_aj)

# converting labels to tensors
train_l_aj = get_tensor_labels(train_l_aj)
val_l_aj = get_tensor_labels(val_l_aj)
test_l_aj = get_tensor_labels(test_l_aj)

Example of dataset after tokenization, encoding, and converting to tensors

In [340]:
train_id_aj[0:5]

tensor([[ 2,  3,  4,  5,  4,  6,  7,  8,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [24, 25, 26, 27, 28, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 2, 30, 31,  6, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 50, 55, 56,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0]])

In [341]:
train_l_aj[0:5]

tensor([0, 1, 0, 0, 1])

Create dataset and dataloader objects from tensors of data. Dataloader object will be used in training.

In [342]:
random_seed = 42
train_dataset = TensorDataset(train_id_aj, train_l_aj)
train_sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(random_seed))

val_dataset = TensorDataset(val_id_aj, val_l_aj)
test_dataset = TensorDataset(test_id_aj, test_l_aj)

batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [357]:
import torchtext.vocab as vocab
class CNN(nn.Module):
    def __init__(self, embed_dim, filter_sizes, num_filters, num_classes, dropout, pretrained_embeddings, vocab_size=len(word2idx)):
        super(CNN, self).__init__()
        if pretrained_embeddings is not None:
            print("using pretrained embedding")
            glove = vocab.GloVe(name='6B', dim=embed_dim)
            self.embedding = nn.Embedding.from_pretrained(glove.vectors, freeze=True)
        else:
            print("training embedding")
            self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=0)
        self.conv1d_list = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels=embed_dim, out_channels=num_filters[i], kernel_size=filter_sizes[i]),
                nn.BatchNorm1d(num_filters[i])
            )
            for i in range(len(filter_sizes))
        ])
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        x_embed = self.embedding(input_ids).float()
        x_reshaped = x_embed.permute(0, 2, 1)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        logits = self.fc(self.dropout(x_fc))
        return logits

In [344]:
class Train:
  def __init__(
      self,
      embed_dim=100,
      filter_sizes=[3, 4, 5],
      num_filters=[150, 150, 150],
      dropout=0.2,
      num_classes=2,
      lr=0.01,
      rho=0.95,
      epoch=5,
      pretrained_embeddings=None,
      weight_decay = 1e-5
    ):
    self._embed_dim = embed_dim
    self._filter_sizes = filter_sizes
    self._num_filters = num_filters
    self._dropout = dropout
    self._num_classes = num_classes
    self._lr = lr
    self._rho = rho
    self._epoch = epoch
    self._pretrained_embeddings = pretrained_embeddings
    self._weight_decay = weight_decay

  def train_model(self, device, train_dataloader, validation_dataloader):
    # Instantiate CNN model
    model = CNN(embed_dim=self._embed_dim,
                filter_sizes=self._filter_sizes,
                num_filters=self._num_filters,
                num_classes=self._num_classes,
                dropout=self._dropout,
                pretrained_embeddings=self._pretrained_embeddings)

    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate SDG Optimizer
    optimizer = optim.SGD(model.parameters(), lr=self._lr, weight_decay=self._weight_decay)

    # cross entropy loss function
    loss_fn = nn.CrossEntropyLoss()

    # Start training loop
    print("Start training...\n")
    best_validation_loss = float('inf')
    best_validation_macro = 0
    best_model = None

    for epoch_i in range(self._epoch):
        total_train_loss = 0
        total_validation_loss = 0
        y_true_t = []
        y_pred_t = []

        y_true_v = []
        y_pred_v = []

        # Put the model into the training mode
        model.train()
        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()
            logits = model(b_input_ids)
            loss = loss_fn(logits, b_labels)
            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            y_true_t.extend(b_labels.cpu().numpy())
            y_pred_t.extend(logits.argmax(1).cpu().numpy())

        # Calculate the average training loss
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Calculate accuracy and F1-macro for the training data
        train_accuracy = accuracy_score(y_true_t, y_pred_t)
        train_f1_macro = f1_score(y_true_t, y_pred_t, average='macro')

        # Put the model into evaluation mode
        model.eval()
        with torch.no_grad():
            for step, batch in enumerate(validation_dataloader):
                b_input_ids, b_labels = tuple(t.to(device) for t in batch)
                logits = model(b_input_ids)
                loss = loss_fn(logits, b_labels)
                total_validation_loss += loss.item()
                y_true_v.extend(b_labels.cpu().numpy())
                y_pred_v.extend(logits.argmax(1).cpu().numpy())

        # Calculate the average validation loss
        avg_validation_loss = total_validation_loss / len(validation_dataloader)

        # Calculate accuracy and F1-macro for the validation data
        val_accuracy = accuracy_score(y_true_v, y_pred_v)
        val_f1_macro = f1_score(y_true_v, y_pred_v, average='macro')

        if val_accuracy > best_validation_macro:
            best_validation_macro = val_accuracy
            best_model = model.state_dict()


        print(f"Epoch {epoch_i + 1:2d} | Train Loss: {avg_train_loss:.6f} | Train Acc: {train_accuracy:.4f} | Train F1-Macro: {train_f1_macro:.4f}")
        print(f"Epoch {epoch_i + 1:2d} | Validation Loss: {avg_validation_loss:.6f} | Validation Acc: {val_accuracy:.4f} | Validation F1-Macro: {val_f1_macro:.4f}")
        print(".........................................................................................")

    # Load the best model for further evaluation or testing
    model.load_state_dict(best_model)
    return model

In [345]:
def test_model(model, device, test_dataloader):
  model.eval()
  y_true = []
  y_pred = []

  with torch.no_grad():
      for step, batch in enumerate(test_dataloader):
          b_input_ids, b_labels = tuple(t.to(device) for t in batch)
          logits = model(b_input_ids)
          y_true.extend(b_labels.cpu().numpy())
          y_pred.extend(logits.argmax(1).cpu().numpy())

  test_accuracy = accuracy_score(y_true, y_pred)
  test_f1_macro = f1_score(y_true, y_pred, average='macro')

  print(f"Test Accuracy: {test_accuracy:.4f}")
  print(f"Test F1-Macro: {test_f1_macro:.4f}")

In [346]:
from sklearn.metrics import f1_score, accuracy_score

# use the GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


### Experiments on different Hyperparameter

In [349]:
# NOT REPORTED
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 3, 4, 5], num_filters=[100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 0.994029 | Train Acc: 0.5089 | Train F1-Macro: 0.4837
Epoch  1 | Validation Loss: 0.691754 | Validation Acc: 0.5469 | Validation F1-Macro: 0.5445
.........................................................................................
Epoch  2 | Train Loss: 0.845472 | Train Acc: 0.5164 | Train F1-Macro: 0.5163
Epoch  2 | Validation Loss: 0.701251 | Validation Acc: 0.5375 | Validation F1-Macro: 0.5363
.........................................................................................
Epoch  3 | Train Loss: 0.851305 | Train Acc: 0.5118 | Train F1-Macro: 0.5117
Epoch  3 | Validation Loss: 0.700473 | Validation Acc: 0.5375 | Validation F1-Macro: 0.5357
.........................................................................................
Epoch  4 | Train Loss: 0.834736 | Train Acc: 0.5196 | Train F1-Macro: 0.5196
Epoch  4 | Validation Loss: 0.699454 | Validation Acc: 0.5375 | Validation F1-Macro: 0.5357
........

In [350]:
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.5, weight_decay=0.00001, filter_sizes=[3, 4, 5], num_filters=[100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 1.072349 | Train Acc: 0.5154 | Train F1-Macro: 0.5123
Epoch  1 | Validation Loss: 0.727376 | Validation Acc: 0.5406 | Validation F1-Macro: 0.5382
.........................................................................................
Epoch  2 | Train Loss: 1.012160 | Train Acc: 0.5179 | Train F1-Macro: 0.5178
Epoch  2 | Validation Loss: 0.749698 | Validation Acc: 0.5406 | Validation F1-Macro: 0.5406
.........................................................................................
Epoch  3 | Train Loss: 0.993536 | Train Acc: 0.5254 | Train F1-Macro: 0.5253
Epoch  3 | Validation Loss: 0.747979 | Validation Acc: 0.5312 | Validation F1-Macro: 0.5312
.........................................................................................
Epoch  4 | Train Loss: 1.008120 | Train Acc: 0.5243 | Train F1-Macro: 0.5241
Epoch  4 | Validation Loss: 0.748104 | Validation Acc: 0.5406 | Validation F1-Macro: 0.5404
........

In [351]:
# MODEL IS OVERFITTING
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.001, dropout=0.2, weight_decay=0.000001, filter_sizes=[3, 3, 4, 5], num_filters=[100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 0.827017 | Train Acc: 0.5271 | Train F1-Macro: 0.5270
Epoch  1 | Validation Loss: 0.711638 | Validation Acc: 0.5094 | Validation F1-Macro: 0.4869
.........................................................................................
Epoch  2 | Train Loss: 0.747559 | Train Acc: 0.5746 | Train F1-Macro: 0.5746
Epoch  2 | Validation Loss: 0.721975 | Validation Acc: 0.5281 | Validation F1-Macro: 0.5202
.........................................................................................
Epoch  3 | Train Loss: 0.700405 | Train Acc: 0.6014 | Train F1-Macro: 0.6014
Epoch  3 | Validation Loss: 0.712928 | Validation Acc: 0.5437 | Validation F1-Macro: 0.5391
.........................................................................................
Epoch  4 | Train Loss: 0.655253 | Train Acc: 0.6371 | Train F1-Macro: 0.6371
Epoch  4 | Validation Loss: 0.714881 | Validation Acc: 0.5437 | Validation F1-Macro: 0.5372
........

In [352]:
# BEST HYPERPArAMETERS
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.5, weight_decay=0.001, filter_sizes=[3, 3, 4, 4, 5], num_filters=[100, 100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 1.131005 | Train Acc: 0.5046 | Train F1-Macro: 0.5008
Epoch  1 | Validation Loss: 0.704471 | Validation Acc: 0.5750 | Validation F1-Macro: 0.5746
.........................................................................................
Epoch  2 | Train Loss: 1.058034 | Train Acc: 0.5193 | Train F1-Macro: 0.5190
Epoch  2 | Validation Loss: 0.724114 | Validation Acc: 0.5719 | Validation F1-Macro: 0.5718
.........................................................................................
Epoch  3 | Train Loss: 1.061867 | Train Acc: 0.5154 | Train F1-Macro: 0.5150
Epoch  3 | Validation Loss: 0.722918 | Validation Acc: 0.5687 | Validation F1-Macro: 0.5681
.........................................................................................
Epoch  4 | Train Loss: 1.040483 | Train Acc: 0.5129 | Train F1-Macro: 0.5127
Epoch  4 | Validation Loss: 0.721993 | Validation Acc: 0.5594 | Validation F1-Macro: 0.5562
........

In [353]:
#### NOT REPORTED
train_obj = Train(embed_dim=50, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 3, 4, 5], num_filters=[150, 150, 150, 150])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 0.859515 | Train Acc: 0.5204 | Train F1-Macro: 0.5183
Epoch  1 | Validation Loss: 0.753476 | Validation Acc: 0.5156 | Validation F1-Macro: 0.5122
.........................................................................................
Epoch  2 | Train Loss: 0.833441 | Train Acc: 0.5229 | Train F1-Macro: 0.5225
Epoch  2 | Validation Loss: 0.769097 | Validation Acc: 0.5156 | Validation F1-Macro: 0.5122
.........................................................................................
Epoch  3 | Train Loss: 0.853287 | Train Acc: 0.5129 | Train F1-Macro: 0.5127
Epoch  3 | Validation Loss: 0.770157 | Validation Acc: 0.5156 | Validation F1-Macro: 0.5122
.........................................................................................
Epoch  4 | Train Loss: 0.827330 | Train Acc: 0.5250 | Train F1-Macro: 0.5246
Epoch  4 | Validation Loss: 0.764839 | Validation Acc: 0.5125 | Validation F1-Macro: 0.5087
........

In [354]:
train_obj = Train(embed_dim=50, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.2, weight_decay=0.00001, filter_sizes=[3, 4, 5], num_filters=[150, 150, 150])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 0.906150 | Train Acc: 0.4818 | Train F1-Macro: 0.4816
Epoch  1 | Validation Loss: 0.720526 | Validation Acc: 0.5406 | Validation F1-Macro: 0.5403
.........................................................................................
Epoch  2 | Train Loss: 0.889799 | Train Acc: 0.4821 | Train F1-Macro: 0.4817
Epoch  2 | Validation Loss: 0.728167 | Validation Acc: 0.5375 | Validation F1-Macro: 0.5372
.........................................................................................
Epoch  3 | Train Loss: 0.893631 | Train Acc: 0.4843 | Train F1-Macro: 0.4843
Epoch  3 | Validation Loss: 0.726136 | Validation Acc: 0.5437 | Validation F1-Macro: 0.5412
.........................................................................................
Epoch  4 | Train Loss: 0.888807 | Train Acc: 0.4614 | Train F1-Macro: 0.4607
Epoch  4 | Validation Loss: 0.724337 | Validation Acc: 0.5281 | Validation F1-Macro: 0.5265
........

### Best Parameter Combination

embed_dim=300,

pretrained_embeddings=True,

epoch=50,

lr=0.0001,

dropout=0.5,

weight_decay=0.001,

filter_sizes=[3, 3, 4, 4, 5],

num_filters=[100, 100, 100, 100, 100]

## Section 2: Emotional Recognition Model: Anger or Sadness

In this section, we will choose data from anger (0) and sadness (3) classes and train a Emotion Recognition model using the best hyperparamters we received in previous section.

In [355]:
from sklearn.metrics import f1_score, accuracy_score

# select Anger and Sadness data
train_d_as = [data for data, label in zip(train_data, train_labels) if label in ('0', '3')]
train_l_as = [label for label in train_labels if label in ('0', '3')]
train_l_as = ["1" if value == "3" else value for value in train_l_as]

val_d_as = [data for data, label in zip(val_data, val_labels) if label in ('0', '3')]
val_l_as = [label for label in val_labels if label in ('0', '3')]
val_l_as = ["1" if value == "3" else value for value in val_l_as]

test_d_as = [data for data, label in zip(test_data, test_labels) if label in ('0', '3')]
test_l_as = [label for label in test_labels if label in ('0', '3')]
test_l_as = ["1" if value == "3" else value for value in test_l_as]

# balance data
train_d_as, train_l_as =  balance_dataset(train_d_as, train_l_as)
val_d_as, val_l_as =  balance_dataset(val_d_as, val_l_as)
test_d_as, test_l_as =  balance_dataset(test_d_as, test_l_as)

# tokenizing, encoding, converting to tensors
train_id_as, word2idx = get_tokenized_encoded_ids(train_d_as)
val_id_as, _ = get_tokenized_encoded_ids(val_d_as)
test_id_as, _ = get_tokenized_encoded_ids(test_d_as)

# converting labels to tensors
train_l_as = get_tensor_labels(train_l_as)
val_l_as = get_tensor_labels(val_l_as)
test_l_as = get_tensor_labels(test_l_as)

# prepare dataloader
random_seed = 42
train_dataset = TensorDataset(train_id_as, train_l_as)
train_sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(random_seed))
val_dataset = TensorDataset(val_id_as, val_l_as)
test_dataset = TensorDataset(test_id_as, test_l_as)

batch_size = 64

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [356]:
# train and test the data with best hyperparameter
train_obj = Train(embed_dim=300, pretrained_embeddings=True, epoch=50, lr=0.0001, dropout=0.5, weight_decay=0.001, filter_sizes=[3, 3, 4, 4, 5], num_filters=[100, 100, 100, 100, 100])
model = train_obj.train_model(device, train_dataloader, val_dataloader)
test_model(model, device, test_dataloader)

using pretrained embedding
Start training...

Epoch  1 | Train Loss: 1.124625 | Train Acc: 0.4975 | Train F1-Macro: 0.4945
Epoch  1 | Validation Loss: 0.760941 | Validation Acc: 0.4313 | Validation F1-Macro: 0.4133
.........................................................................................
Epoch  2 | Train Loss: 1.087234 | Train Acc: 0.5021 | Train F1-Macro: 0.5014
Epoch  2 | Validation Loss: 0.792676 | Validation Acc: 0.4344 | Validation F1-Macro: 0.4158
.........................................................................................
Epoch  3 | Train Loss: 1.075612 | Train Acc: 0.5200 | Train F1-Macro: 0.5194
Epoch  3 | Validation Loss: 0.791352 | Validation Acc: 0.4437 | Validation F1-Macro: 0.4287
.........................................................................................
Epoch  4 | Train Loss: 1.004755 | Train Acc: 0.5175 | Train F1-Macro: 0.5171
Epoch  4 | Validation Loss: 0.785908 | Validation Acc: 0.4406 | Validation F1-Macro: 0.4260
........